<a href="https://colab.research.google.com/github/RamonFontelles/idealista/blob/main/idealista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [261]:
import requests
from bs4 import BeautifulSoup

In [262]:
from pandas import DataFrame
from logging import exception
import re

# COMO ACCEDER A LA URL

In [263]:
# 2. Definir la URL y los Headers para simular un navegador
# La URL es la de la casa que quieres analizar.

#url = "https://www.idealista.com/inmueble/107170538/" #1
url = "https://www.idealista.com/inmueble/88733710/"

In [264]:
# ¡Esto es MUY importante! Los headers hacen que nuestra petición parezca
# que viene de un navegador real (Chrome en este caso) y no de un robot.
# Sin esto, Idealista nos bloqueará con un error 403 (Prohibido).
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "es-ES,es;q=0.9,en;q=0.8",
    "cache-control": "max-age=0",
    "cookie": "_pprv=eyJjb25zZW50Ijp7IjAiOnsibW9kZSI6Im9wdC1pbiJ9LCIxIjp7Im1vZGUiOiJvcHQtaW4ifSwiMiI6eyJtb2RlIjoib3B0LWluIn0sIjMiOnsibW9kZSI6Im9wdC1pbiJ9LCI0Ijp7Im1vZGUiOiJvcHQtaW4ifSwiNSI6eyJtb2RlIjoib3B0LWluIn0sIjYiOnsibW9kZSI6Im9wdC1pbiJ9LCI3Ijp7Im1vZGUiOiJvcHQtaW4ifX0sInB1cnBvc2VzIjpudWxsLCJfdCI6Im1sODdsanRmfG01anNvMmhmIn0%3D; _pcid=%7B%22browserId%22%3A%22m5jso2hdvbtfq55r%22%2C%22_t%22%3A%22ml87lk52%7Cm5jso2t2%22%7D; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAE0RXSwH18yBbVAA4A7KgDWAVgDMAH36SAVhEoAmfNJABfIA; ABTasty=uid=y414qq4bt0a149dn&fst=1749216783138&pst=-1&cst=1749216783138&ns=1&pvt=1&pvis=1&th=; galleryHasBeenBoosted=true; askToSaveAlertPopUp=true; utag_main__sn=9; userUUID=bca4b99d-39c3-4391-b379-03fdc27b1869; SESSION=733bddb87ea43a90~9d08fc99-db59-4443-8b87-dd177c345553; contact9d08fc99-db59-4443-8b87-dd177c345553=\"{'maxNumberContactsAllow':10}\"; cookieSearch-1=\"/venta-viviendas/crevillente-alicante/:1750184635582\"; _last_search=officialZone; utag_main__prevEventLink=; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222026-06-17T18%3A41%3A50.369Z%22%7D; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22c0IUCy4I0E17TZnVtdQZ%22%2C%22expiryDate%22%3A%222026-06-17T18%3A41%3A50.370Z%22%7D; datadome=dnToYColvjIyEAcN5tDn_tStDkoVtG2tu4lTBWaT1im6t35B06PKn9gV4vL9Pk04JUDWzDHn6No3qHDcGvsmXHNPIZcwIvm1lKPlU6BRu~tL1EIzrM_PMkMgFt9jdtk5",
    "priority": "u=0, i",
    "referer": "https://www.idealista.com/venta-viviendas/crevillente-alicante/",
    "sec-ch-device-memory": "8",
    "sec-ch-ua": "\"Google Chrome\";v=\"137\", \"Chromium\";v=\"137\", \"Not/A)Brand\";v=\"24\"",
    "sec-ch-ua-arch": "\"\"",
    "sec-ch-ua-full-version-list": "\"Google Chrome\";v=\"137.0.7151.104\", \"Chromium\";v=\"137.0.7151.104\", \"Not/A)Brand\";v=\"24.0.0.0\"",
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-model": "\"Nexus 5\"",
    "sec-ch-ua-platform": "\"Android\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Mobile Safari/537.36"
}

# 3. Hacer la petición a la página web
print("Accediendo a la página de Idealista...")
response = requests.get(url, headers=headers)

Accediendo a la página de Idealista...


# 3. Parsear el HTML con BeautifulSoup


In [265]:
soup = BeautifulSoup(response.content, 'lxml')

In [266]:
# Otra manera de sacar el titulo
titulo =  soup.find("span", {"class":"main-info__title-main"})
titulo.text.strip()

'Chalet pareado en venta en Centro'

In [267]:
# 4. Crear un diccionario vacío para almacenar los datos
datos_inmueble = {}

# 5. Extraer datos principales (Título, Precio, Descripción)
print("Extrayendo datos principales...")
# a) Extraer el Título
# Buscamos la etiqueta <h1> que tiene la clase 'main-info__title-main'
title_element = soup.find("h1").find("span").text
# Limpiamos el texto de espacios extra al principio o al final
#titulo = title_element.text.strip() if title_element else "Título no encontrado"
titulo = title_element
datos_inmueble['titulo'] = titulo
titulo

Extrayendo datos principales...


'Chalet pareado en venta en Centro'

In [268]:
# PRECIO
precio = soup.find("span", {"class":"info-data-price"}).text.strip()
datos_inmueble['precio'] = precio.replace(".", "").replace("€", "").strip()

datos_inmueble['precio']

'230000'

In [269]:
# UBICACIÓN
ubicacion = soup.find("div", {"id":'headerMap'}).text.strip().strip()

# Elimina la palabra "Ubicación" y convierte saltos de línea múltiples en uno solo
limpia = ubicacion.replace("Ubicación", "").replace("\n", ",")

# Elimina comas duplicadas y espacios redundantes
datos_inmueble['ubicacion'] = ','.join([x.strip() for x in limpia.split(",") if x.strip()])

datos_inmueble['ubicacion']

'Distrito Centro,Santa Pola,Baix Vinalopó,Alicante'

In [270]:
# CARACTERÍSTICAS PRINCIPALES

caracteristicas = soup.find("div", {"class":'info-features'}).text.strip()
caracteristicas = caracteristicas.replace("\n", ",")
caracteristicas = ','.join([x.strip() for x in caracteristicas.split(",") if x.strip()])
print(caracteristicas)
datos_inmueble['m2'] = caracteristicas.split(",")[0][:-3]
datos_inmueble['habitaciones']= caracteristicas.split(",")[1][:-5]
try:
  datos_inmueble['planta'] = caracteristicas.split(",")[2]
except:
  datos_inmueble['planta'] = "No indicado"

datos_inmueble['m2'], datos_inmueble['habitaciones'], datos_inmueble['planta']

90 m²,5 hab.


('90', '5', 'No indicado')

In [271]:
# Para la descripción, buscamos el contenedor y luego el párrafo dentro
descripcion_container = soup.find('div', class_='adCommentsLanguage')
if descripcion_container:
    # Usamos .replace() para convertir los <br/> en saltos de línea legibles
    datos_inmueble['descripcion'] = descripcion_container.find('p').get_text(separator='\n').strip()
datos_inmueble['descripcion']

'IDEALPISOS A. P. I. (Agente de la Propiedad Inmobiliaria) Nº 01151, les presenta esta casa unifamiliar ubicada en Santa Pola, a tan solo unos minutos del Supermercado Eroski y demás comercios.\nCuenta con unas vistas exteriores y despejadas al colegio, pabellón y parques infantiles.\nLa vivienda consta de 2 Plantas de 87 m² + terraza que se distribuyen de la siguiente manera:\n-En la planta baja tiene una vivienda de unos 70 m², dispone de salón/comedor, cocina, un baño con plato de ducha y 1 habitación + patio, donde se puede hacer otra habitación si se desea.\n-En la primera planta disponemos de la segunda vivienda la cual está compuesta por:\n3 dormitorios, salón/comedor, un baño completo con plato de ducha y cocina independiente, todas sus estancias disponen de mucha luminosidad ya que la vivienda hace esquina y es muy luminosa.\n-En la última planta tenemos el acceso a la amplia terraza con vistas panorámicas tanto al colegio y como al parque infantil. En la cual se puede constru

In [272]:

# 6. Extraer Características (Básicas, Equipamiento, Certificado)
datos_inmueble['caracteristicas'] = {}

# Buscamos el contenedor principal de todas las características
details_container = soup.find('div', class_='details-property')

if details_container:
    # a) Características básicas
    basic_features_div = details_container.find('div', class_='details-property-feature-one')
    if basic_features_div:
        items = basic_features_div.find_all('li')
        datos_inmueble['caracteristicas']['basicas'] = [item.text.strip() for item in items]

    # b) Equipamiento y Certificado energético
    extra_features_div = details_container.find('div', class_='details-property-feature-two')
    if extra_features_div:
        # Equipamiento
        h2_equipamiento = extra_features_div.find('h2', string='Equipamiento')
        if h2_equipamiento:
            ul_equipamiento = h2_equipamiento.find_next_sibling('div').find('ul')
            items_equip = ul_equipamiento.find_all('li')
            datos_inmueble['caracteristicas']['equipamiento'] = [item.text.strip() for item in items_equip]

        # Certificado energético
        h2_certificado = extra_features_div.find('h2', string='Certificado energético')
        if h2_certificado:
            datos_inmueble['caracteristicas']['certificado_energetico'] = {}
            ul_certificado = h2_certificado.find_next_sibling('div').find('ul')
            # Extraer Consumo
            try:
                consumo_li = ul_certificado.find('span', string='Consumo: ').find_parent('li')
                # El truco: la calificación está en la clase del icono
                clase_consumo = consumo_li.find('span', class_=lambda x: x and x.startswith('icon-energy-')).get('class')[-1]
                calificacion_consumo = clase_consumo.split('-')[-1].upper()
                datos_inmueble['caracteristicas']['certificado_energetico']['consumo'] = calificacion_consumo
            except:
                datos_inmueble['caracteristicas']['certificado_energetico']['consumo'] = "No indicado"
            # Extraer Emisiones
            try:
              emisiones_li = ul_certificado.find('span', string='Emisiones: ').find_parent('li')
              clase_emisiones = emisiones_li.find('span', class_=lambda x: x and x.startswith('icon-energy-')).get('class')[-1]
              calificacion_emisiones = clase_emisiones.split('-')[-1].upper()
              datos_inmueble['caracteristicas']['certificado_energetico']['emisiones'] = calificacion_emisiones
            except:
              datos_inmueble['caracteristicas']['certificado_energetico']['emisiones'] = "No indicado"

# --- FIN DE LA EXTRACCIÓN ---

# 7. Imprimir el diccionario resultante de forma legible
print("\n--- DICCIONARIO CON LOS DATOS EXTRAÍDOS ---")
print(datos_inmueble)


--- DICCIONARIO CON LOS DATOS EXTRAÍDOS ---
{'titulo': 'Chalet pareado en venta en Centro', 'precio': '230000', 'ubicacion': 'Distrito Centro,Santa Pola,Baix Vinalopó,Alicante', 'm2': '90', 'habitaciones': '5', 'planta': 'No indicado', 'descripcion': 'IDEALPISOS A. P. I. (Agente de la Propiedad Inmobiliaria) Nº 01151, les presenta esta casa unifamiliar ubicada en Santa Pola, a tan solo unos minutos del Supermercado Eroski y demás comercios.\nCuenta con unas vistas exteriores y despejadas al colegio, pabellón y parques infantiles.\nLa vivienda consta de 2 Plantas de 87 m² + terraza que se distribuyen de la siguiente manera:\n-En la planta baja tiene una vivienda de unos 70 m², dispone de salón/comedor, cocina, un baño con plato de ducha y 1 habitación + patio, donde se puede hacer otra habitación si se desea.\n-En la primera planta disponemos de la segunda vivienda la cual está compuesta por:\n3 dormitorios, salón/comedor, un baño completo con plato de ducha y cocina independiente, tod

In [273]:
# --- MOSTRAR RESULTADOS COMPLETOS ---

print("\n--- DATOS DEL INMUEBLE ---")
print(f"Título: {datos_inmueble['titulo']}")
print(f"Precio: {datos_inmueble['precio']} €")
print(f"Ubicación: {datos_inmueble['ubicacion']}")
print(f"Habitaciones: {datos_inmueble['habitaciones']}")
print(f"Metros cuadrados: {datos_inmueble['m2']}")
print(f"Plnata: {datos_inmueble['planta']}")

print("\nDescripción:")
print(datos_inmueble['descripcion'])

print("\nCaracterísticas:")
caracteristicas = datos_inmueble.get('caracteristicas', {})

if caracteristicas:
    for clave, valor in caracteristicas.items():
        print(f"\n· {clave.capitalize()}:")
        if isinstance(valor, list):
            for item in valor:
                print(f"  - {item}")
        elif isinstance(valor, dict):
            for subclave, subvalor in valor.items():
                print(f"  · {subclave.capitalize()}: {subvalor}")
        else:
            print(f"  - {valor}")
else:
    print("No se encontraron características.")




--- DATOS DEL INMUEBLE ---
Título: Chalet pareado en venta en Centro
Precio: 230000 €
Ubicación: Distrito Centro,Santa Pola,Baix Vinalopó,Alicante
Habitaciones: 5
Metros cuadrados: 90
Plnata: No indicado

Descripción:
IDEALPISOS A. P. I. (Agente de la Propiedad Inmobiliaria) Nº 01151, les presenta esta casa unifamiliar ubicada en Santa Pola, a tan solo unos minutos del Supermercado Eroski y demás comercios.
Cuenta con unas vistas exteriores y despejadas al colegio, pabellón y parques infantiles.
La vivienda consta de 2 Plantas de 87 m² + terraza que se distribuyen de la siguiente manera:
-En la planta baja tiene una vivienda de unos 70 m², dispone de salón/comedor, cocina, un baño con plato de ducha y 1 habitación + patio, donde se puede hacer otra habitación si se desea.
-En la primera planta disponemos de la segunda vivienda la cual está compuesta por:
3 dormitorios, salón/comedor, un baño completo con plato de ducha y cocina independiente, todas sus estancias disponen de mucha lumi

In [274]:
# 1. Importar las librerías necesarias
from bs4 import BeautifulSoup
import json  # Para procesar los datos de las fotos
import pprint # Para imprimir el diccionario de forma bonita

# 2. El contenido HTML que proporcionaste
# Asegúrate de pegar todo el código HTML del anuncio dentro de las triples comillas.
html_content = """
<!DOCTYPE html>
<html class="" data-userauth="false" env="es" lang="es" username="">
<!-- PEGA AQUÍ EL HTML COMPLETO -->
</html>
"""

# 3. Parsear el HTML con BeautifulSoup
print("Analizando el código HTML del anuncio...")
soup = BeautifulSoup(html_content, 'lxml')

# 4. Crear un diccionario para almacenar los datos
datos_casa = {}

# --- EXTRACCIÓN DE DATOS ---


# b) LINKS DE LAS FOTOS (El método más eficaz)
print("Extrayendo links de las fotos...")
# ¡Aquí está la magia! Los datos de las fotos están en un script, lo que es más fácil de procesar.
scripts = soup.find_all('script')
photo_data_script = None
for script in scripts:
    if script.string and 'adMultimediasInfo' in script.string:
        photo_data_script = script.string
        break

if photo_data_script:
    # Limpiamos el string para que sea un JSON válido
    json_string = photo_data_script.split('adMultimediasInfo = ', 1)[1].strip().rstrip(';')
    multimedia_data = json.loads(json_string)

    # Extraemos las URLs de alta resolución
    fotos = multimedia_data.get('fullScreenGalleryPics', [])
    datos_casa['links_fotos'] = [foto['imageDataService'] for foto in fotos]
else:
    datos_casa['links_fotos'] = ["No se encontraron datos de fotos en el script."]

# c) DATOS ADICIONALES SUGERIDOS
print("Extrayendo datos adicionales sugeridos...")
datos_casa['sugerencias'] = {}

# Descripción del anunciante
descripcion_container = soup.find('div', class_='adCommentsLanguage')
if descripcion_container and descripcion_container.find('p'):
    datos_casa['sugerencias']['descripcion'] = descripcion_container.find('p').get_text(separator='\n').strip()

# Características Básicas
basic_features_div = soup.find('div', class_='details-property-feature-one')
if basic_features_div:
    items = basic_features_div.find_all('li')
    datos_casa['sugerencias']['caracteristicas_basicas'] = [item.text.strip() for item in items]

# Equipamiento
extra_features_div = soup.find('div', class_='details-property-feature-two')
if extra_features_div and extra_features_div.find('h2', string='Equipamiento'):
    ul_equipamiento = extra_features_div.find('h2', string='Equipamiento').find_next_sibling('div').find('ul')
    items_equip = ul_equipamiento.find_all('li')
    datos_casa['sugerencias']['equipamiento'] = [item.text.strip() for item in items_equip]




# --- FIN DE LA EXTRACCIÓN ---

# 5. Imprimir el diccionario resultante de forma legible
print("\n--- DATOS EXTRAÍDOS DEL INMUEBLE ---")
pprint.pprint(datos_casa)

Analizando el código HTML del anuncio...
Extrayendo links de las fotos...
Extrayendo datos adicionales sugeridos...

--- DATOS EXTRAÍDOS DEL INMUEBLE ---
{'links_fotos': ['No se encontraron datos de fotos en el script.'],
 'sugerencias': {}}
